## Classification of Arabic Tweets based on its written dialect

# Import Libraries

In [50]:
import re
import os
import csv
import nltk
import json
import utils
import pickle
import string
import warnings
import requests
import numpy as np
import pandas as pd
import tensorflow as tf
import urllib.request
import matplotlib.pyplot as plt

from time import time

from sklearn.svm import SVC
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report

from keras.layers import Embedding, Dense, Dropout, Input, LSTM, Bidirectional
from keras.layers import MaxPooling1D, Conv1D, Flatten
from keras.preprocessing import sequence, text
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.utils import np_utils
from keras.callbacks import Callback

from gensim.models.keyedvectors import KeyedVectors
  
warnings.filterwarnings(action = 'ignore')
np.random.seed(42)

# Reading csv file

In [2]:
dialect_df = pd.read_csv('dialect_dataset.csv')
id_df =  dialect_df.drop('dialect',axis=1)
dialect_class_df = dialect_df['dialect']

# Retreiving Data

In [3]:
url = 'https://recruitment.aimtechnologies.co/ai-tasks'

list_of_ids_ints = id_df['id'].tolist()
list_of_id_strings = [str(x) for x in list_of_ids_ints]
id_list_max_range = []
id_list_final_range = []
retreived_ids = []
retreived_text = []

for i in range (len(list_of_id_strings)):
    id_list_max_range.append(list_of_id_strings[i])
    if i >= 458000:
        id_list_final_range.append(list_of_id_strings[i])
        print(len(id_list_final_range))
        if i == 458196:
            r = requests.post(url, data= json.dumps(id_list_final_range))
            dialect = r.json()
            for key, value in dialect.items():
                retreived_ids.append(key)
                retreived_text.append(value)
    if ((i+1) % 1000 == 0):
        r = requests.post(url, data= json.dumps(id_list_max_range))
        dialect = r.json()
        for key, value in dialect.items():
            retreived_ids.append(key)
            retreived_text.append(value)
        id_list_max_range = []

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197


In [4]:
retreived_ids_df = pd.DataFrame(retreived_ids, columns =['retreived_ids'])
retreived_text_df = pd.DataFrame(retreived_text, columns =['retreived_text'])
print(retreived_ids_df.iloc[0:10])
print(retreived_text_df.iloc[0:10])
retreived_ids_var = retreived_ids_df["retreived_ids"]
retreived_text_var = retreived_text_df["retreived_text"]
dialect_df = dialect_df.join(retreived_ids_var)
dialect_df = dialect_df.join(retreived_text_var)
print(dialect_df)

         retreived_ids
0  1175358310087892992
1  1175416117793349632
2  1175450108898565888
3  1175471073770573824
4  1175496913145217024
5  1175668034146643968
6  1175670153884983296
7  1175671762580856832
8  1175715664398561280
9  1176019816072777728
                                      retreived_text
0   @Nw8ieJUwaCAAreT لكن بالنهاية .. ينتفض .. يغير .
1  @7zNqXP0yrODdRjK يعني هذا محسوب على البشر .. ح...
2                    @KanaanRema مبين من كلامه خليجي
3         @HAIDER76128900 يسلملي مرورك وروحك الحلوه💐
4                 @hmo2406 وين هل الغيبه  اخ محمد 🌸🌺
5  @Badi9595 @KanaanRema يااخي الإرهابي اذا كان ع...
6  @SarahNadhum90 @nUBNTdfVgACYQxV مطلبي يقدم است...
7  @KanaanRema @Badi9595 خلص والله لعيونكم انا ما...
8  @SalahAlarbawi يمكن سؤال فات الكثير اللي يصور ...
9  @Eng_alow91 @cb4LwpWrS1hT5lb @EdyCohen اولا ان...
                         id dialect        retreived_ids  \
0       1175358310087892992      IQ  1175358310087892992   
1       1175416117793349632      IQ  1175416

# Preprocessing

In [5]:
lower_string = retreived_text[0].lower()
no_number_string = re.sub(r'\d+','',lower_string)
print(no_number_string)
no_punc_string = re.sub(r'[^\w\s]','', no_number_string)
print(no_punc_string)
no_wspace_string = no_punc_string.strip()
print(no_wspace_string)
text = re.sub('([@A-Za-z0-9_]+)|[^\w\s]|#|http\S+', '', no_wspace_string) # cleaning up
print (text)
after = " ".join(text.split())
print(after)

@nwiejuwacaaret لكن بالنهاية .. ينتفض .. يغير .
nwiejuwacaaret لكن بالنهاية  ينتفض  يغير 
nwiejuwacaaret لكن بالنهاية  ينتفض  يغير
 لكن بالنهاية  ينتفض  يغير
لكن بالنهاية ينتفض يغير


In [6]:
retreived_text_cleaned = []
for i in range (len(retreived_text)):
    lower_string = retreived_text[i].lower()
    no_number_string = re.sub(r'\d+','',lower_string)
    no_punc_string = re.sub(r'[^\w\s]','', no_number_string)
    no_wspace_string = no_punc_string.strip()
    text = re.sub('([@A-Za-z0-9_]+)|[^\w\s]|#|http\S+', '', no_wspace_string)
    text = re.sub(r'(.)\1+', r'\1', text)
    after = " ".join(text.split())
    retreived_text_cleaned.append(after)
retreived_text_cleaned_df = pd.DataFrame(retreived_text_cleaned, columns =['clean_text'])

In [7]:
# The first step is to subject the data to preprocessing.
# This involves removing both arabic and english punctuation
# Normalizing different letter variants with one common letter
# first we define a list of arabic and english punctiations that we want to get rid of in our text
punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' + string.punctuation
# Arabic stop words with nltk
stop_words = stopwords.words()
arabic_diacritics = re.compile("""
                             ّ    | # Shadda
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
def preprocess(text):
    '''
    text is an arabic string input
    
    the preprocessed text is returned
    '''   
    #remove punctuations
    translator = str.maketrans('', '', punctuations)
    text = text.translate(translator)
    # remove Tashkeel
    text = re.sub(arabic_diacritics, '', text)
    #remove longation
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text
retreived_text_cleaned_df['clean_text'] = retreived_text_cleaned_df['clean_text'].apply(preprocess)

In [8]:
final_df = pd.DataFrame(retreived_text_cleaned_df, columns = ['clean_text'])
final_df = final_df.join(dialect_class_df)
final_df.head(20)

,clean_text,dialect
0,بالنهايه ينتفض يغير,IQ
1,يعني محسوب علي البشر حيونه وحشيه وتطلبون الغرب...,IQ
2,مبين كلامه خليجي,IQ
3,يسلملي مرورك وروحك الحلوه,IQ
4,وين الغيبه اخ محمد,IQ
5,ياخي الارهابي اذا عراقي سعودي فلسطيني وين المش...,IQ
6,مطلبي يقدم استقالته وفوكاها اعتذار,IQ
7,خلص واله لعيونكم انا ماعندي شيء معه بالعكس متا...,IQ
8,يمكن سءال فات الكثير الي يصور شنو موقفه وكانه ...,IQ
9,اولا اني ردت علي رجل جنوبي والي ذكر حجابها ثان...,IQ


# Split Dataset into training and testing sets

In [15]:
X = final_df.clean_text
y = final_df.dialect
le = LabelEncoder()
y = le.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [24]:
clean_train_text_list = X_train.tolist()
clean_test_text_list = X_test.tolist()

In [25]:
# tfidf = TfidfVectorizer()
# # get tf-df values
# result = tfidf.fit_transform(clean_train_text_list)  
# # get idf values
# print('\nidf values:')
# for ele1, ele2 in zip(tfidf.get_feature_names(), tfidf.idf_):
#     print(ele1, ':', ele2)    
# # get indexing
# print('\nWord indexes:')
# print(tfidf.vocabulary_)  
# # display tf-idf values
# print('\ntf-idf value:')
# print(result)  
# # in matrix form
# print('\ntf-idf values in matrix form:')
# print(result.toarray())

# TF-IDF with Logistic Regression

In [51]:
# make pipeline
pipe = make_pipeline(TfidfVectorizer(),
                    LogisticRegression())
# make param grid
param_grid = {'logisticregression__C': [0.01, 0.1, 1, 10, 100]}

# create and fit the model
model_lr = GridSearchCV(pipe, param_grid, cv=5)
model_lr.fit(X_train,y_train)

# make prediction and print accuracy
prediction = model_lr.predict(X_test)

In [52]:
print(model_lr.best_score_)
print(model_lr.best_estimator_)
print(model_lr.best_params_)

0.4955836201013787
Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('logisticregression', LogisticRegression(C=10))])
{'logisticregression__C': 10}


In [53]:
print(f"Accuracy score is {accuracy_score(y_test, prediction):.2f}")
print(classification_report(y_test, prediction))

Accuracy score is 0.50
              precision    recall  f1-score   support

           0       0.40      0.39      0.39      7863
           1       0.34      0.30      0.32      7956
           2       0.59      0.47      0.52      4864
           3       0.68      0.79      0.73     17143
           4       0.61      0.50      0.55      4691
           5       0.41      0.31      0.35      8316
           6       0.42      0.58      0.49     12603
           7       0.60      0.64      0.62      8258
           8       0.54      0.68      0.60     10946
           9       0.83      0.50      0.62      3558
          10       0.39      0.30      0.34      5775
          11       0.43      0.53      0.48     13141
          12       0.45      0.45      0.45      9380
          13       0.37      0.39      0.38      7961
          14       0.68      0.53      0.60      4435
          15       0.45      0.28      0.34      4882
          16       0.72      0.32      0.44      2830
    

# Deploy LR model

In [54]:
model_lr_final = model_lr.fit(X,y)

In [83]:
# Saving the Python objects as serialized files can be done using pickle library
# Here let us save the Final ZomatoRatingModel
with open('model_lr_final.pkl', 'wb') as fileWriteStream:
    pickle.dump(model_lr_final, fileWriteStream)
    # Don't forget to close the filestream!
    fileWriteStream.close()
    
print('pickle file of Predictive Model is saved at Location:',os.getcwd())

pickle file of Predictive Model is saved at Location: C:\Users\mohamed mohssen\AIM Task


In [84]:
# This Function can be called from any from any front end tool/website
def FunctionPredictResult(InputData):
    Num_Inputs=InputData
    
    # Making sure the input data has same columns as it was used for training the model
    # Also, if standardization/normalization was done, then same must be done for new input
    
    # Appending the new data with the Training data
#     DataForML=pd.read_pickle('DataForML.pkl')
#     InputData=InputData.append(DataForML)
    
    # Generating dummy variables for rest of the nominal variables
    InputData=pd.get_dummies(InputData)
            
    # Maintaining the same order of columns as it was during the model training
    Predictors=['clean_text']
    
    # Generating the input values to the model
    X=InputData[Predictors].values[0:Num_Inputs]
    
    # Generating the standardized values of X since it was done while model training also
#     X=PredictorScalerFit.transform(X)
    
    # Loading the Function from pickle file
    import pickle
    with open('model_lr_final.pkl', 'rb') as fileReadStream:
        PredictionModel=pickle.load(fileReadStream)
        # Don't forget to close the filestream!
        fileReadStream.close()
            
    # Genrating Predictions
    Prediction=PredictionModel.predict(X)
    PredictionResult=pd.DataFrame(Prediction, columns=['Prediction'])
    return(PredictionResult)

In [87]:
# # Calling the function for new sample data
# vectorizer = TfidfVectorizer()
# X = vectorizer.fit_transform("هي دي الحياة")
# NewSampleData=pd.DataFrame(
# data=X,
# columns=['clean_text'])

# print(NewSampleData)

# # Calling the Function for prediction
# FunctionPredictResult(InputData= NewSampleData)

In [88]:
# # Creating the function which can take inputs and return predictions
# def FunctionGeneratePrediction(test_tweet):
    
#     # Creating a data frame for the model input
#     SampleInputData=pd.DataFrame(
#      data=[[test_tweet]],
#      columns=['clean_text'])

#     # Calling the function defined above using the input parameters
#     Predictions=FunctionPredictResult(InputData= SampleInputData)

#     # Returning the prediction
#     return(Predictions.to_json())

# # Function call
# FunctionGeneratePrediction( [12 34 45 56] )

# LSTM Model with Embedding

In [28]:
vocab_size = 56
encoded_train_tweets = [one_hot(d, vocab_size) for d in clean_train_text_list]
encoded_test_tweets = [one_hot(d, vocab_size) for d in clean_test_text_list]
# pad documents to a max length of n words
max_length = 18
padded_train_tweets = pad_sequences(encoded_train_tweets, maxlen=max_length, padding='post')
padded_test_tweets = pad_sequences(encoded_test_tweets, maxlen=max_length, padding='post')
print(padded_train_tweets)

[[25 26 48 ...  0  0  0]
 [51 25 12 ...  0  0  0]
 [49 25 36 ...  0  0  0]
 ...
 [30 53 37 ...  0  0  0]
 [30 44 37 ...  0  0  0]
 [14 42 55 ... 48 27 36]]


# Calculating f1-score

In [76]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# Deep Learning Model structure

In [77]:
embedding_vector_features= 18
model=Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=max_length))
model.add(LSTM(128,input_shape=(padded_train_tweets.shape),activation='relu',return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256,activation='relu'))
model.add(Dropout(0.2))
# for units in [128,128,64,32]:
# model.add(Dense(units,activation='relu'))
# model.add(Dropout(0.2))
model.add(Dense(56,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(18,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc',f1_m,precision_m, recall_m],optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3))
print(model.summary())
# optimizer='adam'

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 18, 18)            1008      
_________________________________________________________________
lstm_9 (LSTM)                (None, 18, 128)           75264     
_________________________________________________________________
dropout_14 (Dropout)         (None, 18, 128)           0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 256)               394240    
_________________________________________________________________
dropout_15 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 56)                14392     
_________________________________________________________________
dropout_16 (Dropout)         (None, 56)               

# Model Fit

In [78]:
model.fit(padded_train_tweets,y_train,validation_data=(padded_test_tweets,y_test),epochs=5,batch_size=32)

Epoch 1/5
10024/10024 [==============================] - 418s 41ms/step - loss: 2.7416 - acc: 0.1415 - f1_m: 11.6584 - precision_m: 1684991360.0000 - recall_m: 5.8376 - val_loss: 2.7298 - val_acc: 0.1470 - val_f1_m: 11.7380 - val_precision_m: 1751960320.0000 - val_recall_m: 5.8710
Epoch 2/5
10024/10024 [==============================] - 413s 41ms/step - loss: 2.7274 - acc: 0.1481 - f1_m: 11.5424 - precision_m: 1623100032.0000 - recall_m: 5.7850 - val_loss: 2.7212 - val_acc: 0.1496 - val_f1_m: 11.5526 - val_precision_m: 1736542976.0000 - val_recall_m: 5.7770
Epoch 3/5
10024/10024 [==============================] - 418s 42ms/step - loss: 2.7222 - acc: 0.1489 - f1_m: 11.4765 - precision_m: 1594533760.0000 - recall_m: 5.7542 - val_loss: 2.7191 - val_acc: 0.1493 - val_f1_m: 11.4543 - val_precision_m: 1686156032.0000 - val_recall_m: 5.7318
Epoch 4/5
10024/10024 [==============================] - 421s 42ms/step - loss: 2.7186 - acc: 0.1506 - f1_m: 11.4573 - precision_m: 1566105728.0000 - reca

# Model Evaluation

In [80]:
# evaluate the model
loss, accuracy, f1_score, precision, recall = model.evaluate(padded_test_tweets, y_test, verbose=1)

4296/4296 [==============================] - 63s 15ms/step - loss: 2.7139 - acc: 0.1504 - f1_m: 11.3860 - precision_m: 1654795648.0000 - recall_m: 5.6999


In [82]:
print(f1_score)
print(accuracy)

11.385997772216797
0.15038557350635529
